In [74]:
#import all libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import copy
from collections import namedtuple, deque
import gym
import matplotlib.pyplot as plt
import random
%matplotlib inline
import argparse
import gym_duckietown
from gym_duckietown.envs import DuckietownEnv
from gym_duckietown.simulator import Simulator
from gym_duckietown.wrappers import UndistortWrapper



INFO:gym-duckietown:Using gym-duckietown version 2018.8.2-more-exposure

INFO:gym-duckietown:Registering gym environment id: Duckietown-4way-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_obstacles-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-udem1-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-zigzag_dists-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_pedestrians-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_dyn_duckiebots-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-straight_road-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-small_loop-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_empty-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-small_loop_cw-v0


In [96]:
def launch_env(id=None):
    env = None
    if id is None:
        from gym_duckietown.simulator import Simulator
        env = Simulator(
            seed=123, # random seed
            map_name="loop_empty",
            max_steps=500001, # we don't want the gym to reset itself
            domain_rand=0,
            camera_width=640,
            camera_height=480,
            accept_start_angle_deg=4, # start close to straight
            full_transparency=True,
            distortion=True,
        )
    else:
        env = gym.make(id)

return env


UnboundLocalError: local variable 'env' referenced before assignment

In [86]:
#initialise actor network
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

class Actor(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_dim, action_dim, max_action):
       
        super(Actor, self).__init__()
        flat_size = 32 * 9 * 14

        self.lr = nn.LeakyReLU()
        self.tanh = nn.Tanh()
        self.sigm = nn.Sigmoid()

        self.conv1 = nn.Conv2d(3, 32, 8, stride=2)
        self.conv2 = nn.Conv2d(32, 32, 4, stride=2)
        self.conv3 = nn.Conv2d(32, 32, 4, stride=2)
        self.conv4 = nn.Conv2d(32, 32, 4, stride=1)

        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(32)
        self.bn4 = nn.BatchNorm2d(32)

        self.dropout = nn.Dropout(.5)

        self.lin1 = nn.Linear(flat_size, 512)
        self.lin2 = nn.Linear(512, action_dim)

        self.max_action = max_action

    def forward(self, x):
        x = self.bn1(self.lr(self.conv1(x)))
        x = self.bn2(self.lr(self.conv2(x)))
        x = self.bn3(self.lr(self.conv3(x)))
        x = self.bn4(self.lr(self.conv4(x)))
        x = x.view(x.size(0), -1)  # flatten
        x = self.dropout(x)
        x = self.lr(self.lin1(x))
         # because we don't want our duckie to go backwards
        x = self.lin2(x)
        x[:, 0] = self.max_action * self.sigm(x[:, 0])  # because we don't want the duckie to go backwards
        x[:, 1] = self.tanh(x[:, 1])

        return x



class Critic(nn.Module):
    """Critic (Value) Model."""

    def __init__(self,state_dim, action_dim, max_action):
        super(Critic, self).__init__()
        flat_size = 32 * 9 * 14

        self.lr = nn.LeakyReLU()

        self.conv1 = nn.Conv2d(3, 32, 8, stride=2)
        self.conv2 = nn.Conv2d(32, 32, 4, stride=2)
        self.conv3 = nn.Conv2d(32, 32, 4, stride=2)
        self.conv4 = nn.Conv2d(32, 32, 4, stride=1)

        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(32)
        self.bn4 = nn.BatchNorm2d(32)

        self.dropout = nn.Dropout(.5)

        self.lin1 = nn.Linear(flat_size, 256)
        self.lin2 = nn.Linear(256 + action_dim, 128)
        self.lin3 = nn.Linear(128, 1)

    def forward(self, states, actions):
        x = self.bn1(self.lr(self.conv1(states)))
        x = self.bn2(self.lr(self.conv2(x)))
        x = self.bn3(self.lr(self.conv3(x)))
        x = self.bn4(self.lr(self.conv4(x)))
        x = x.view(x.size(0), -1)  # flatten
        x = self.lr(self.lin1(x))
        x = self.lr(self.lin2(torch.cat([x, actions], 1)))  # c
        x = self.lin3(x)

        return x


In [87]:
class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.2):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(len(x))])
        self.state = x + dx
        return self.state

class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
     
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)
        

In [105]:
class Agent():
    """Interacts with and learns from the environment."""
    
    def __init__(self, state_size, action_size,max_action, random_seed):
       
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(random_seed)

        # Actor Network (w/ Target Network)
        self.actor_local = Actor(state_size, action_size,max_action).to(device)
        self.actor_target = Actor(state_size, action_size, max_action).to(device)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters(), lr=LR_ACTOR)

        # Critic Network (w/ Target Network)
        self.critic_local = Critic(state_size, action_size, max_action).to(device)
        self.critic_target = Critic(state_size, action_size,max_action).to(device)
        self.critic_optimizer = optim.Adam(self.critic_local.parameters(), lr=LR_CRITIC, weight_decay=WEIGHT_DECAY)

        # Noise process
        self.noise = OUNoise(action_size, random_seed)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, random_seed)
    
    def step(self, state, action, reward, next_state, done):
        """Save experience in replay memory, and use random sample from buffer to learn."""
        # Save experience / reward
        self.memory.add(state, action, reward, next_state, done)

        # Learn, if enough samples are available in memory
        if len(self.memory) > BATCH_SIZE:
            experiences = self.memory.sample()
            self.learn(experiences, GAMMA)

    def act(self, state, add_noise=True):
        """Returns actions for given state as per current policy."""
        state = torch.from_numpy(state).float().to(device)
        state = state.unsqueeze(0)
        self.actor_local.eval()
        with torch.no_grad():
            action = self.actor_local(state).cpu().data.numpy()
        self.actor_local.train()
        if add_noise:
            action += self.noise.sample()
        return np.clip(action, -1, 1)

    def reset(self):
        self.noise.reset()

    def learn(self, experiences, gamma):
       
        states, actions, rewards, next_states, dones = experiences

        # ---------------------------- update critic ---------------------------- #
        # Get predicted next-state actions and Q values from target models
        actions_next = self.actor_target(next_states)
        Q_targets_next = self.critic_target(next_states, actions_next)
        # Compute Q targets for current states (y_i)
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        # Compute critic loss
        Q_expected = self.critic_local(states, actions)
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # ---------------------------- update actor ---------------------------- #
        # Compute actor loss
        actions_pred = self.actor_local(states)
        actor_loss = -self.critic_local(states, actions_pred).mean()
        # Minimize the loss
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # ----------------------- update target networks ----------------------- #
        self.soft_update(self.critic_local, self.critic_target, TAU)
        self.soft_update(self.actor_local, self.actor_target, TAU)                     

    def soft_update(self, local_model, target_model, tau):
       
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)



In [106]:
#wrappers
import gym
from gym import spaces
import numpy as np


class ResizeWrapper(gym.ObservationWrapper):
    def __init__(self, env=None, shape=(120, 160, 3)):
        super(ResizeWrapper, self).__init__(env)
        self.observation_space.shape = shape
        self.observation_space = spaces.Box(
            self.observation_space.low[0, 0, 0],
            self.observation_space.high[0, 0, 0],
            shape,
            dtype=self.observation_space.dtype)
        self.shape = shape

    def observation(self, observation):
        from scipy.misc import imresize
        return imresize(observation, self.shape)


class NormalizeWrapper(gym.ObservationWrapper):
    def __init__(self, env=None):
        super(NormalizeWrapper, self).__init__(env)
        self.obs_lo = self.observation_space.low[0, 0, 0]
        self.obs_hi = self.observation_space.high[0, 0, 0]
        obs_shape = self.observation_space.shape
        self.observation_space = spaces.Box(0.0, 1.0, obs_shape, dtype=np.float32)

    def observation(self, obs):
        if self.obs_lo == 0.0 and self.obs_hi == 1.0:
            return obs
        else:
            return (obs - self.obs_lo) / (self.obs_hi - self.obs_lo)


class ImgWrapper(gym.ObservationWrapper):
    def __init__(self, env=None):
        super(ImgWrapper, self).__init__(env)
        obs_shape = self.observation_space.shape
        self.observation_space = spaces.Box(
            self.observation_space.low[0, 0, 0],
            self.observation_space.high[0, 0, 0],
            [obs_shape[2], obs_shape[0], obs_shape[1]],
            dtype=self.observation_space.dtype)

    def observation(self, observation):
        return observation.transpose(2, 0, 1)


class DtRewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super(DtRewardWrapper, self).__init__(env)

    def reward(self, reward):
        if reward == -1000:
            reward = -10
        elif reward > 0:
            reward += 10
        else:
            reward += 4

        return reward


# this is needed because at max speed the duckie can't turn anymore
class ActionWrapper(gym.ActionWrapper):
    def __init__(self, env):
        super(ActionWrapper, self).__init__(env)

    def action(self, action):
        action_ = [action[0] * 0.8, action[1]]
        return action_


In [115]:
env = gym.make('Duckietown-udem1-v0')
env.seed(1); torch.manual_seed(1);
env = ResizeWrapper(env)
env = NormalizeWrapper(env)
env = ImgWrapper(env) # to make the images from 160x120x3 into 3x160x120
# env = ActionWrapper(env)
env = DtRewardWrapper(env)

DEBUG:gym-duckietown:loading map file "/home/sharath/gym-duckietown/gym_duckietown/maps/udem1.yaml"
INFO:gym-duckietown:Starting at [ 3.54450865  0.          1.02821116] 3.650690357079699
INFO:gym-duckietown:using DuckietownEnv


In [119]:
env.seed(2)
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 128        # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR_ACTOR = 1e-4         # learning rate of the actor 
LR_CRITIC = 1e-3        # learning rate of the critic
WEIGHT_DECAY = 0        # L2 weight decay


state_size = env.observation_space.shape
action_size = env.action_space.shape[0]
max_action = float(env.action_space.high[0])
print(state_size)
print(action_size)
agent = Agent(state_size=state_size, action_size=action_size,max_action = max_action, random_seed=2)


def ddpg(n_episodes=1000, max_t=300, print_every=100):
    scores_deque = deque(maxlen=print_every)
    scores = []
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        agent.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state)
            print(action[0])
            next_state, reward, done, _ = env.step(action[0])
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_deque.append(score)
        scores.append(score)
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
        torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
        torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            
    return scores

scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

INFO:gym-duckietown:Starting at [ 3.99277878  0.          2.5096587 ] 2.085982766594644


(3, 120, 160)
2
[ 0.68740451  0.22027734]
[ 0.67003411  0.20883548]
[ 0.81105101  0.32938057]
[ 0.89777064  0.34611151]
[ 0.95870519  0.42011461]
[ 1.          0.39345747]
[ 1.          0.41775021]
[ 1.          0.55864632]
[ 1.          0.58817971]
[ 1.          0.55813676]
[ 1.         0.4844622]
[ 1.         0.4800992]
[ 1.          0.59120184]
[ 1.          0.61923486]
[ 1.          0.53573382]


INFO:gym-duckietown:Stopping the simulator because we are at an invalid pose.
INFO:gym-duckietown:Starting at [ 2.62041211  0.          0.9392967 ] 4.188762868634082


Episode 1	Average Score: 26.69[ 0.56124991  0.05792566]
[ 0.65353674  0.25357485]
[ 0.76483548  0.25650856]
[ 0.90322858  0.38196075]
[ 0.98906749  0.51048231]
[ 1.          0.59662545]


INFO:gym-duckietown:Stopping the simulator because we are at an invalid pose.


...

INFO:gym-duckietown:Stopping the simulator because we are at an invalid pose.
INFO:gym-duckietown:Starting at [ 1.0188516   0.          3.08006769] 5.60895792440526


[ 1.         0.5324105]
[ 1.       0.49348]
[ 0.98916274  0.46754274]
[ 1.         0.4948937]
Episode 5	Average Score: 73.15[ 0.55812311  0.15890472]
[ 0.59132564  0.32100412]
[ 0.76966524  0.4232451 ]
[ 0.8154074   0.46672204]
[ 0.88375944  0.41142312]
[ 0.90922946  0.45952359]
[ 0.8835299   0.41392732]
[ 0.98591387  0.42974254]
[ 1.          0.55408978]
[ 1.          0.53349507]
[ 1.          0.53247494]
[ 1.          0.59429944]
[ 1.          0.57099551]
[ 1.         0.6244632]
[ 1.          0.62568063]
[ 1.          0.63346046]
[ 0.98049819  0.66078359]
[ 0.92264646  0.62306976]
[ 0.93327522  0.72128755]
[ 0.88298607  0.76882511]
[ 0.86346501  0.77235615]


INFO:gym-duckietown:Stopping the simulator because we are at an invalid pose.
INFO:gym-duckietown:Starting at [ 1.02536221  0.          1.34784621] 1.7817166954469845


[ 0.88673317  0.75375926]
[ 0.97888207  0.72422194]
[ 0.93082047  0.64455098]
Episode 6	Average Score: 81.49[ 0.51226467  0.20071946]
[ 0.63809162  0.36702543]
[ 0.75533146  0.32156807]
[ 0.84826988  0.43342429]
[ 0.94016671  0.47262865]
[ 0.94508433  0.49783245]
[ 1.          0.48153758]
[ 1.          0.50932527]
[ 1.         0.5984962]
[ 1.          0.68052381]
[ 1.         0.6292513]
[ 1.          0.59135646]
[ 1.          0.64310849]
[ 1.          0.66844261]


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [32, 3, 8, 8], but got input of size [384, 120, 160] instead